# https://note.com/npaka/n/nc70d33764018 参照

In [11]:
!pip install langchain openai tiktoken elasticsearch

  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/15/02/f0d7c68ed90594d90891ee13b87d621ffc3434cccbc461d1f72086ebf0e1/tiktoken-0.5.1-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for regex>=2022.1.18 from https://files.pythonhosted.org/packages/27/b8/fde0e99442b328d159bd0b2c0ff5401e1f1839e7a8d7339308b3915c7faa/regex-2023.10.3-cp311-cp311-macosx_10_9_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 847.7 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.5/953.5 kB 4.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 4.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OPENAI_API_KEY")

Enter your OPENAI_API_KEY ········


In [58]:
from langchain.document_loaders import TextLoader

loader = TextLoader("./demo.md")
document = loader.load()
document

[Document(page_content='LangChainは、大規模言語モデル (LLM)を使用してアプリケーションの作成を簡素化するように設計されたフレームワークです。言語モデル統合フレームワークとしての LangChain のユースケースは、ドキュメント分析と要約、チャットボット、コード分析など、一般的な言語モデルのユースケースと大きく重複します。\n', metadata={'source': './demo.md'})]

In [47]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import MarkdownTextSplitter

loader = TextLoader("./rock.md")
documents = loader.load()
text_splitter = MarkdownTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(documents)
docs

[Document(page_content='# ぼっちざろっく\n## あらすじ\n### 結束バンド\n後藤ひとりは友達を作れない陰キャでいつも一人で過ごしていたが、中学時代にテレビのインタビューを見て、陰キャでもバンドを組んでいれば人気者になれると聞き、ギターの練習を始める。ひとりは毎日コツコツ練習を重ねることでギターの腕前はかなりのものとなっていたが、気がつけば中学は卒業。結局、友達を作るという当初の目標は達成することはできずじまいだった。そして高校に入学したひとりはギター演奏を動画配信し、「ギターヒーロー」としてネットでそこそこの人気を集めていたが、現実では変わらず友達を作れずにいた。ひとりはギターをこれみよがしに持ち、ギタリストだということをアピールするものの効果はなし。黄昏（たそがれ）ていたところ、ギタリストを探していた伊地知虹夏に誘われ、バンド活動をすることとなる。ひとりは虹夏、山田リョウらと共に「結束バンド」として活動を始めるが、友達は欲しいけどコミュ症独りぼっちのひとりは問題行動ばかり起こしていた。結束バンドなのに、結束力ゼロな凸凹バンド活動は幕開けするのだった。そしてそんなひとりに、クラスメイトの喜多郁代が話しかけてきて、彼女が結束バンドの元メンバーだったことを知る。紆余（うよ）曲折の末、郁代はひとりとの交流で結束バンドに戻ることになり、ひとりたちはライブに向けて準備をすることとなる。ひとりはライブチケットのノルマをこなすため、方々を回ってチケットを売ることになるが、人見知りのひとりには難題ですぐに壁にぶち当たってしまう。絶望するひとりは、そこで行き倒れた廣井きくりと遭遇。彼女の計らいで突発的な路上ライブを行なって、観客を魅了する。きくりのお陰でライブチケットのノルマもこなし、結束バンドはライブに臨むのだった。', metadata={'source': './rock.md'}),
 Document(page_content='### 文化祭ライブ\n夏休みに入り、後藤ひとりは知り合いも増えていたが、自分から遊びに誘うことができずに夏休みも終わり、新学期を迎えていた。相も変わらずコミュ症なひとりは文化祭で盛り上がるクラスメイトをよそに、一人黄昏ていた。そんな中、みんなの前で結束バンドのライブをすればクラスで人気者になれるかもと考えたひとり

In [74]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import ElasticsearchStore

embeddings = OpenAIEmbeddings()
url = f"http://localhost:9200"
index_name="demo"

db = ElasticsearchStore(
    embedding=embeddings,
    index_name=index_name,
    es_url=url,
    strategy=ElasticsearchStore.ExactRetrievalStrategy()
)

In [ ]:
db.add_documents(docs)

In [61]:
db.add_documents(document)

['29932539-8a52-458a-b34e-2a46f643f723']

In [82]:
# Example of a custom query thats just doing a BM25 search on the text field.
def custom_query(query_body: dict, query: str):
    """Custom query to be used in Elasticsearch.
    Args:
        query_body (dict): Elasticsearch query body.
        query (str): Query string.
    Returns:
        dict: Elasticsearch query body.
    """
    #print("Query Retriever created by the retrieval strategy:")
    #print(query_body)
    #print()

    new_query_body = {"query": {"match": {"text": query}}}

    #print("Query thats actually used in Elasticsearch:")
    #print(new_query_body)
    print()

    return new_query_body


In [84]:
db.client.indices.refresh(index=index_name)

query = "ぼっちざろっくの主人公"
results = db.similarity_search(query, custom_query=custom_query, k=3)
print(results)


[Document(page_content='# ぼっちざろっく\n## あらすじ\n### 結束バンド\n後藤ひとりは友達を作れない陰キャでいつも一人で過ごしていたが、中学時代にテレビのインタビューを見て、陰キャでもバンドを組んでいれば人気者になれると聞き、ギターの練習を始める。ひとりは毎日コツコツ練習を重ねることでギターの腕前はかなりのものとなっていたが、気がつけば中学は卒業。結局、友達を作るという当初の目標は達成することはできずじまいだった。そして高校に入学したひとりはギター演奏を動画配信し、「ギターヒーロー」としてネットでそこそこの人気を集めていたが、現実では変わらず友達を作れずにいた。ひとりはギターをこれみよがしに持ち、ギタリストだということをアピールするものの効果はなし。黄昏（たそがれ）ていたところ、ギタリストを探していた伊地知虹夏に誘われ、バンド活動をすることとなる。ひとりは虹夏、山田リョウらと共に「結束バンド」として活動を始めるが、友達は欲しいけどコミュ症独りぼっちのひとりは問題行動ばかり起こしていた。結束バンドなのに、結束力ゼロな凸凹バンド活動は幕開けするのだった。そしてそんなひとりに、クラスメイトの喜多郁代が話しかけてきて、彼女が結束バンドの元メンバーだったことを知る。紆余（うよ）曲折の末、郁代はひとりとの交流で結束バンドに戻ることになり、ひとりたちはライブに向けて準備をすることとなる。ひとりはライブチケットのノルマをこなすため、方々を回ってチケットを売ることになるが、人見知りのひとりには難題ですぐに壁にぶち当たってしまう。絶望するひとりは、そこで行き倒れた廣井きくりと遭遇。彼女の計らいで突発的な路上ライブを行なって、観客を魅了する。きくりのお陰でライブチケットのノルマもこなし、結束バンドはライブに臨むのだった。', metadata={'source': './rock.md'}), Document(page_content='### 喜多 郁代 (きた いくよ)\n秀華高校に通う女子。後藤ひとりとは同学年。ウェーブの入った赤い髪をセミロングに伸ばした美少女で、社交的な性格をしている。いわゆる「陽キャ」で、ひとりとは正反対で要領がよく、友達も多い。一方でなんでも無難にこなせるゆえに特技といったものもなく、自分を平凡な人間と思っているのがコ

In [162]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo",model_kwargs={'stream':False})
qa = RetrievalQAWithSourcesChain.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever(search_kwargs={'k': 1}), return_source_documents=True)

In [ ]:
#Agent

In [146]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import Tool

tools = [
    Tool(
      name = "elasticsearch_search",
      func=qa,
      description="テキストデータをロードしたelasticsearchから情報検索する"
  )
]

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)


In [147]:
from langchain.chains.qa_with_sources.map_reduce_prompt import QUESTION_PROMPT
from langchain import PromptTemplate

template = """
下記の質問に日本語で答えてください。
質問：{question}
回答：
"""

prompt = PromptTemplate(
    input_variables=["question"],
    template=template,
)


In [148]:
query = "大槻ヨヨコはどのバンドのリーダーですか？"
question = prompt.format(question=query)

agent.run(question)




> Entering new AgentExecutor chain...
どのバンドのリーダーかを検索する必要があります。
Action: elasticsearch_search
Action Input: "大槻ヨヨコ バンド リーダー"
Observation: {'question': '大槻ヨヨコ バンド リーダー', 'answer': '大槻ヨヨコは人気メタルバンド「SIDEROS」のリーダーであり、ギター兼ボーカルを担当しています。\n', 'sources': './rock.md'}
Thought:大槻ヨヨコは人気メタルバンド「SIDEROS」のリーダーであり、ギター兼ボーカルを担当しています。
Final Answer: 大槻ヨヨコは「SIDEROS」のリーダーです。

> Finished chain.


'大槻ヨヨコは「SIDEROS」のリーダーです。'

In [163]:
result=qa("ぼっちちゃんの得意な楽器は？")

In [164]:
result

{'question': 'ぼっちちゃんの得意な楽器は？',
 'answer': 'ぼっちちゃんの得意な楽器はギターです。\n',
 'sources': './rock.md',
 'source_documents': [Document(page_content='# ぼっちざろっく\n## あらすじ\n### 結束バンド\n後藤ひとりは友達を作れない陰キャでいつも一人で過ごしていたが、中学時代にテレビのインタビューを見て、陰キャでもバンドを組んでいれば人気者になれると聞き、ギターの練習を始める。ひとりは毎日コツコツ練習を重ねることでギターの腕前はかなりのものとなっていたが、気がつけば中学は卒業。結局、友達を作るという当初の目標は達成することはできずじまいだった。そして高校に入学したひとりはギター演奏を動画配信し、「ギターヒーロー」としてネットでそこそこの人気を集めていたが、現実では変わらず友達を作れずにいた。ひとりはギターをこれみよがしに持ち、ギタリストだということをアピールするものの効果はなし。黄昏（たそがれ）ていたところ、ギタリストを探していた伊地知虹夏に誘われ、バンド活動をすることとなる。ひとりは虹夏、山田リョウらと共に「結束バンド」として活動を始めるが、友達は欲しいけどコミュ症独りぼっちのひとりは問題行動ばかり起こしていた。結束バンドなのに、結束力ゼロな凸凹バンド活動は幕開けするのだった。そしてそんなひとりに、クラスメイトの喜多郁代が話しかけてきて、彼女が結束バンドの元メンバーだったことを知る。紆余（うよ）曲折の末、郁代はひとりとの交流で結束バンドに戻ることになり、ひとりたちはライブに向けて準備をすることとなる。ひとりはライブチケットのノルマをこなすため、方々を回ってチケットを売ることになるが、人見知りのひとりには難題ですぐに壁にぶち当たってしまう。絶望するひとりは、そこで行き倒れた廣井きくりと遭遇。彼女の計らいで突発的な路上ライブを行なって、観客を魅了する。きくりのお陰でライブチケットのノルマもこなし、結束バンドはライブに臨むのだった。', metadata={'source': './rock.md'})]}

In [161]:
from ragas.langchain.evalchain import RagasEvaluatorChain
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_precision,
    context_recall,
)

# create evaluation chains
faithfulness_chain = RagasEvaluatorChain(metric=faithfulness)
answer_rel_chain = RagasEvaluatorChain(metric=answer_relevancy)
context_rel_chain = RagasEvaluatorChain(metric=context_precision)
context_recall_chain = RagasEvaluatorChain(metric=context_recall)

In [167]:
ragas_result={
    'query': result['question'],
    'result': result['answer'],
    'source_documents': result['source_documents']
}
ragas_result

{'query': 'ぼっちちゃんの得意な楽器は？',
 'result': 'ぼっちちゃんの得意な楽器はギターです。\n',
 'source_documents': [Document(page_content='# ぼっちざろっく\n## あらすじ\n### 結束バンド\n後藤ひとりは友達を作れない陰キャでいつも一人で過ごしていたが、中学時代にテレビのインタビューを見て、陰キャでもバンドを組んでいれば人気者になれると聞き、ギターの練習を始める。ひとりは毎日コツコツ練習を重ねることでギターの腕前はかなりのものとなっていたが、気がつけば中学は卒業。結局、友達を作るという当初の目標は達成することはできずじまいだった。そして高校に入学したひとりはギター演奏を動画配信し、「ギターヒーロー」としてネットでそこそこの人気を集めていたが、現実では変わらず友達を作れずにいた。ひとりはギターをこれみよがしに持ち、ギタリストだということをアピールするものの効果はなし。黄昏（たそがれ）ていたところ、ギタリストを探していた伊地知虹夏に誘われ、バンド活動をすることとなる。ひとりは虹夏、山田リョウらと共に「結束バンド」として活動を始めるが、友達は欲しいけどコミュ症独りぼっちのひとりは問題行動ばかり起こしていた。結束バンドなのに、結束力ゼロな凸凹バンド活動は幕開けするのだった。そしてそんなひとりに、クラスメイトの喜多郁代が話しかけてきて、彼女が結束バンドの元メンバーだったことを知る。紆余（うよ）曲折の末、郁代はひとりとの交流で結束バンドに戻ることになり、ひとりたちはライブに向けて準備をすることとなる。ひとりはライブチケットのノルマをこなすため、方々を回ってチケットを売ることになるが、人見知りのひとりには難題ですぐに壁にぶち当たってしまう。絶望するひとりは、そこで行き倒れた廣井きくりと遭遇。彼女の計らいで突発的な路上ライブを行なって、観客を魅了する。きくりのお陰でライブチケットのノルマもこなし、結束バンドはライブに臨むのだった。', metadata={'source': './rock.md'})]}

In [168]:
faithfulness_chain(ragas_result)

{'query': 'ぼっちちゃんの得意な楽器は？',
 'result': 'ぼっちちゃんの得意な楽器はギターです。\n',
 'source_documents': [Document(page_content='# ぼっちざろっく\n## あらすじ\n### 結束バンド\n後藤ひとりは友達を作れない陰キャでいつも一人で過ごしていたが、中学時代にテレビのインタビューを見て、陰キャでもバンドを組んでいれば人気者になれると聞き、ギターの練習を始める。ひとりは毎日コツコツ練習を重ねることでギターの腕前はかなりのものとなっていたが、気がつけば中学は卒業。結局、友達を作るという当初の目標は達成することはできずじまいだった。そして高校に入学したひとりはギター演奏を動画配信し、「ギターヒーロー」としてネットでそこそこの人気を集めていたが、現実では変わらず友達を作れずにいた。ひとりはギターをこれみよがしに持ち、ギタリストだということをアピールするものの効果はなし。黄昏（たそがれ）ていたところ、ギタリストを探していた伊地知虹夏に誘われ、バンド活動をすることとなる。ひとりは虹夏、山田リョウらと共に「結束バンド」として活動を始めるが、友達は欲しいけどコミュ症独りぼっちのひとりは問題行動ばかり起こしていた。結束バンドなのに、結束力ゼロな凸凹バンド活動は幕開けするのだった。そしてそんなひとりに、クラスメイトの喜多郁代が話しかけてきて、彼女が結束バンドの元メンバーだったことを知る。紆余（うよ）曲折の末、郁代はひとりとの交流で結束バンドに戻ることになり、ひとりたちはライブに向けて準備をすることとなる。ひとりはライブチケットのノルマをこなすため、方々を回ってチケットを売ることになるが、人見知りのひとりには難題ですぐに壁にぶち当たってしまう。絶望するひとりは、そこで行き倒れた廣井きくりと遭遇。彼女の計らいで突発的な路上ライブを行なって、観客を魅了する。きくりのお陰でライブチケットのノルマもこなし、結束バンドはライブに臨むのだった。', metadata={'source': './rock.md'})],
 'faithfulness_score': 1.0}

In [169]:
answer_rel_chain(ragas_result)

{'query': 'ぼっちちゃんの得意な楽器は？',
 'result': 'ぼっちちゃんの得意な楽器はギターです。\n',
 'source_documents': [Document(page_content='# ぼっちざろっく\n## あらすじ\n### 結束バンド\n後藤ひとりは友達を作れない陰キャでいつも一人で過ごしていたが、中学時代にテレビのインタビューを見て、陰キャでもバンドを組んでいれば人気者になれると聞き、ギターの練習を始める。ひとりは毎日コツコツ練習を重ねることでギターの腕前はかなりのものとなっていたが、気がつけば中学は卒業。結局、友達を作るという当初の目標は達成することはできずじまいだった。そして高校に入学したひとりはギター演奏を動画配信し、「ギターヒーロー」としてネットでそこそこの人気を集めていたが、現実では変わらず友達を作れずにいた。ひとりはギターをこれみよがしに持ち、ギタリストだということをアピールするものの効果はなし。黄昏（たそがれ）ていたところ、ギタリストを探していた伊地知虹夏に誘われ、バンド活動をすることとなる。ひとりは虹夏、山田リョウらと共に「結束バンド」として活動を始めるが、友達は欲しいけどコミュ症独りぼっちのひとりは問題行動ばかり起こしていた。結束バンドなのに、結束力ゼロな凸凹バンド活動は幕開けするのだった。そしてそんなひとりに、クラスメイトの喜多郁代が話しかけてきて、彼女が結束バンドの元メンバーだったことを知る。紆余（うよ）曲折の末、郁代はひとりとの交流で結束バンドに戻ることになり、ひとりたちはライブに向けて準備をすることとなる。ひとりはライブチケットのノルマをこなすため、方々を回ってチケットを売ることになるが、人見知りのひとりには難題ですぐに壁にぶち当たってしまう。絶望するひとりは、そこで行き倒れた廣井きくりと遭遇。彼女の計らいで突発的な路上ライブを行なって、観客を魅了する。きくりのお陰でライブチケットのノルマもこなし、結束バンドはライブに臨むのだった。', metadata={'source': './rock.md'})],
 'answer_relevancy_score': 0.894759063627628}

In [170]:
context_rel_chain(ragas_result)

{'query': 'ぼっちちゃんの得意な楽器は？',
 'result': 'ぼっちちゃんの得意な楽器はギターです。\n',
 'source_documents': [Document(page_content='# ぼっちざろっく\n## あらすじ\n### 結束バンド\n後藤ひとりは友達を作れない陰キャでいつも一人で過ごしていたが、中学時代にテレビのインタビューを見て、陰キャでもバンドを組んでいれば人気者になれると聞き、ギターの練習を始める。ひとりは毎日コツコツ練習を重ねることでギターの腕前はかなりのものとなっていたが、気がつけば中学は卒業。結局、友達を作るという当初の目標は達成することはできずじまいだった。そして高校に入学したひとりはギター演奏を動画配信し、「ギターヒーロー」としてネットでそこそこの人気を集めていたが、現実では変わらず友達を作れずにいた。ひとりはギターをこれみよがしに持ち、ギタリストだということをアピールするものの効果はなし。黄昏（たそがれ）ていたところ、ギタリストを探していた伊地知虹夏に誘われ、バンド活動をすることとなる。ひとりは虹夏、山田リョウらと共に「結束バンド」として活動を始めるが、友達は欲しいけどコミュ症独りぼっちのひとりは問題行動ばかり起こしていた。結束バンドなのに、結束力ゼロな凸凹バンド活動は幕開けするのだった。そしてそんなひとりに、クラスメイトの喜多郁代が話しかけてきて、彼女が結束バンドの元メンバーだったことを知る。紆余（うよ）曲折の末、郁代はひとりとの交流で結束バンドに戻ることになり、ひとりたちはライブに向けて準備をすることとなる。ひとりはライブチケットのノルマをこなすため、方々を回ってチケットを売ることになるが、人見知りのひとりには難題ですぐに壁にぶち当たってしまう。絶望するひとりは、そこで行き倒れた廣井きくりと遭遇。彼女の計らいで突発的な路上ライブを行なって、観客を魅了する。きくりのお陰でライブチケットのノルマもこなし、結束バンドはライブに臨むのだった。', metadata={'source': './rock.md'})],
 'context_precision_score': 0.0}